# Automatic generation of Notebook using PyCropML
    This notebook implements a crop model.

### Model Cumulttfrom

In [ ]:
MODULE list_sub
    IMPLICIT NONE
    TYPE container
        CLASS(*), ALLOCATABLE :: item
        CLASS(*), ALLOCATABLE :: items(:)
    END TYPE

    interface Add
        module procedure AddToListFloat
        module procedure AddToListInt
        module procedure AddToListChar
        module procedure AddToListArray
    end interface
CONTAINS

    FUNCTION indice(vectorElem, elem)
        CHARACTER(LEN=*), DIMENSION(:):: vectorElem
        INTEGER::iterator, indice
        CHARACTER(LEN=*):: elem
        DO iterator= 1, SIZE(vectorElem)
            IF(vectorElem(iterator)==elem) THEN
                indice = iterator
            END IF
        END DO
        RETURN
    END FUNCTION indice


    SUBROUTINE AddToListFloat(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        REAL, INTENT(IN) :: element
        REAL, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        REAL, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListFloat

    SUBROUTINE AddToListInt(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        INTEGER, INTENT(IN) :: element
        INTEGER, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        INTEGER, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListInt

    SUBROUTINE AddToListChar(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize, l
        CHARACTER(LEN=*), INTENT(IN) :: element
        CHARACTER(LEN=*), DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        CHARACTER(LEN=65), DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            l=1
            ALLOCATE(list(l))
            list(l) = element
        END IF
    END SUBROUTINE AddToListChar


    SUBROUTINE AddToListArray(a, e)
        TYPE(container),ALLOCATABLE,INTENT(INOUT) :: a(:)
        CLASS(*),INTENT(IN), allocatable :: e(:)
        TYPE(container),ALLOCATABLE :: tmp(:)

        IF (.NOT.ALLOCATED(a)) THEN
            ALLOCATE(a(1))
            ALLOCATE(a(1)%items(SIZE(e)), source = e)
        ELSE
            CALL MOVE_ALLOC(a,tmp)
            ALLOCATE(a(SIZE(tmp)+1))
            a(1:SIZE(tmp)) = tmp
            ALLOCATE(a(SIZE(tmp)+1)%items(SIZE(e)), source = e)
        END IF
    END SUBROUTINE AddToListArray

END MODULE list_sub
MODULE Cumulttfrommod
    USE list_sub
    IMPLICIT NONE
CONTAINS
    SUBROUTINE model_cumulttfrom(calendarMoments, &
        calendarCumuls, &
        cumulTT, &
        cumulTTFromZC_65, &
        cumulTTFromZC_39, &
        cumulTTFromZC_91)
        CHARACTER(65), ALLOCATABLE , DIMENSION(:), INTENT(IN) ::  &
                calendarMoments
        REAL, ALLOCATABLE , DIMENSION(:), INTENT(IN) :: calendarCumuls
        REAL, INTENT(IN) :: cumulTT
        REAL, INTENT(OUT) :: cumulTTFromZC_65
        REAL, INTENT(OUT) :: cumulTTFromZC_39
        REAL, INTENT(OUT) :: cumulTTFromZC_91
        !- Description:
    !            - Model Name: CumulTTFrom Model
    !            - Author: Pierre Martre
    !            - Reference: Modeling development phase in the 
    !                Wheat Simulation Model SiriusQuality.
    !                See documentation at http://www1.clermont.inra.fr/siriusquality/?page_id=427
    !            - Institution: INRA Montpellier
    !            - Abstract: Calculate CumulTT 
        !- inputs:
    !            - name: calendarMoments
    !                          - description : List containing appearance of each stage
    !                          - variablecategory : state
    !                          - datatype : STRINGLIST
    !                          - default : ['Sowing']
    !                          - unit : 
    !                          - inputtype : variable
    !            - name: calendarCumuls
    !                          - description : list containing for each stage occured its cumulated thermal times
    !                          - variablecategory : state
    !                          - datatype : DOUBLELIST
    !                          - default : [0.0]
    !                          - unit : °C d
    !                          - inputtype : variable
    !            - name: cumulTT
    !                          - description : cumul TT at current date
    !                          - datatype : DOUBLE
    !                          - variablecategory : auxiliary
    !                          - min : -200
    !                          - max : 10000
    !                          - default : 8
    !                          - unit : °C d
    !                          - inputtype : variable
        !- outputs:
    !            - name: cumulTTFromZC_65
    !                          - description :  cumul TT from Anthesis to current date 
    !                          - variablecategory : auxiliary
    !                          - datatype : DOUBLE
    !                          - min : 0
    !                          - max : 5000
    !                          - unit : °C d
    !            - name: cumulTTFromZC_39
    !                          - description :  cumul TT from FlagLeafLiguleJustVisible to current date 
    !                          - variablecategory : auxiliary
    !                          - datatype : DOUBLE
    !                          - min : 0
    !                          - max : 5000
    !                          - unit : °C d
    !            - name: cumulTTFromZC_91
    !                          - description :  cumul TT from EndGrainFilling to current date 
    !                          - variablecategory : auxiliary
    !                          - datatype : DOUBLE
    !                          - min : 0
    !                          - max : 5000
    !                          - unit : °C d
        cumulTTFromZC_65 = 0.0
        cumulTTFromZC_39 = 0.0
        cumulTTFromZC_91 = 0.0
        IF(ANY(calendarMoments .EQ. 'Anthesis')) THEN
            cumulTTFromZC_65 = cumulTT - calendarCumuls(indice(calendarMoments,  &
                    'Anthesis'))
        END IF
        IF(ANY(calendarMoments .EQ. 'FlagLeafLiguleJustVisible')) THEN
            cumulTTFromZC_39 = cumulTT - calendarCumuls(indice(calendarMoments,  &
                    'FlagLeafLiguleJustVisible'))
        END IF
        IF(ANY(calendarMoments .EQ. 'EndGrainFilling')) THEN
            cumulTTFromZC_91 = cumulTT - calendarCumuls(indice(calendarMoments,  &
                    'EndGrainFilling'))
        END IF
    END SUBROUTINE model_cumulttfrom

END MODULE
PROGRAM test
    USE Cumulttfrommod
    CHARACTER(65), ALLOCATABLE, DIMENSION(:):: calendarMoments
    REAL, ALLOCATABLE, DIMENSION(:):: calendarCumuls
    REAL:: cumulTT
    REAL:: cumulTTFromZC_65
    REAL:: cumulTTFromZC_39
    REAL:: cumulTTFromZC_91
    print *, "--------test_test_wheat1_CumulTTFrom-------"
    CALL Add(calendarMoments, "Sowing")
    CALL Add(calendarMoments, "Emergence")
    CALL Add(calendarMoments, "FloralInitiation")
    CALL Add(calendarMoments, "FlagLeafLiguleJustVisible")
    CALL Add(calendarMoments, "Heading")
    CALL Add(calendarMoments, "Anthesis")
    calendarCumuls = [0.0, 112.330110409888, 354.582294511779,  &
            741.510096671757, 853.999637026622, 954.59002776961]
    cumulTT = 972.970888983105
    call model_cumulttfrom(calendarMoments, calendarCumuls, cumulTT,  &
            cumulTTFromZC_65, cumulTTFromZC_39, cumulTTFromZC_91)
    !cumulTTFromZC_65: 18.38
    print *, "cumulTTFromZC_65 estimated :"
    print *, cumulTTFromZC_65
    !cumulTTFromZC_39: 231.46
    print *, "cumulTTFromZC_39 estimated :"
    print *, cumulTTFromZC_39
    !cumulTTFromZC_91: 0
    print *, "cumulTTFromZC_91 estimated :"
    print *, cumulTTFromZC_91

END PROGRAM
